<a href="https://colab.research.google.com/github/kocelifk/GNN-Implementation/blob/main/recommendation_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Social recommendation system can be represented as user usual social graph and user item graph.



in our project our problem is to predict the rating as a part of the recommendation system so here we have tried to implement the basic recommendation system using the social network structure.



so this overall architecture having three main components the left one is the user modeling and the item modeling and the rating prediction.

in user modeling the aim is to learn the user latent factor. the challenge here how to inherently combine the user item graph and the social graph because in user modeling ou have two kind of thing.


one is the item aggregation another is the social aggregation.








in item aggregation it is utilized to learn item space user latent Vector from the user item graph and the second type of aggregation which is social aggregation where social space is a latent vector is learned from the graph
then finally we concatenate these two latent factor and we get one user latent factor from the user model on the other hand in the item modeling which is shown in the right part of this architecture.









we do something called user aggregation here we use a similar method as learning
item space usual latent factor via item aggregation, so we will come to the detail path while I will walk through on the core part.


So let me walk through on the code part so here I have built one collab notebook which is having three GPU access so the first step is to import these required packages as I am running in the collab notebook I
have uploaded the data file in the drive itself so let's execute that cell then I am changing this directory to whatever folder I have saved all the folders everything.





this is the system architecture, overall system architecture.
Now let's load the data first. So for the data set we have taken
epinions data set(https://www.kaggle.com/datasets/masoud3/epinions-trust-network) which is publicly available data set and it can be got from the kaggle website also it is a real world data set which is widely used to evaluate many of the recommender system in the liquid literature.




in this data set we have two sort of information one is the user ratings on on the items another is the explicit trust or distrust relationship between users.


So as you can see after loading the rating.mat file we are getting some rating information so this kind of Matrix and other is the we are getting the trust information like the friend ID.











Then we are just trying to prepare to list one is the user item list, another
the user user list also let me just ah execute some of the sale
here.


So we are preparing training validation and testing sample by some splitting ninety percent ten percent and ten percent as the split ratio. So we have trading samples of 7300054 and the validation sample have the same number of inputs.


Now we are shaping this data into pickle file for loading in future.








then let me show you some of the data frame how it looks like. So in the
training data frame, we have three kind of label one is the user ID another is the item ID and as the label so here label is the rating.


So for an example user ID 1 given rating 3 that is the label 3 to the item id 8. In the same way if you see the last row, user ID 22166 gave rating 5 to the item id 53125.












now we have all the information from the rating Matrix file is ready.

so we can have 27 types of ratings starting from 1 to 27 that I have already shown here the maximum level and the minimum level in these two cell next I am trying to prepare one user item list.











so as you can see the in the user item lists what I am doing I am just adding the item ID and the corresponding rating for each of the user and that is
creating a list.


For an example while this cell is running let me show
you that user item list index number 3 is showing for the third user what are the corresponding item and the corresponding rating so for the third user he has given rating to item number 57 to rating 15 item number 52 rating 10 like that in the same way I have also prepared the item user list.







so in the item user list for each of the item I have saved which
who is the user give the what relative for that particular item
for an example let me show you on the second index of this item user lists.

 so for the second item user ID 9729 gave rating 2 and likewise user number 9789 keep rating two for that particular item second second item.




so this way I have prepared the item user list. It will take some time to just execute this cell.

Okay while it is running I will just show you the code of to process the trust information as well so in case of trust information I am creating a data frame.





so who are the friends for the particular user id 2. So here you can see
in the example so for the user id 2 we have so many friends like 7659 is the
friends. So from the trust Matrix information Matrix we can get those
informations.






so language from the trust information
whatever ah
least we need like user item list and
the user users item list we have saved
in a pickle file when we load the
training loader or validation loader
those people files will just reload so
this is a one time
activity once the pickle file is saved
we do not have to do any kind of pre
processing so till this part it is kind
of data preprocessing of the opinions
trust network data set
let me check how it is okay 48 percent


so
here I have prepared one grab data set
class basic graph data set Plus
where the user item list and user user
list and the users item lists also is
taken
so after preparing the grab data set
class
there is one more function we need that
is correlate function so in any problem
like in NLP problem also when we are
just trying to process any sequence
there should be some length maximum
length that we can process so here also
for each of the user there might be
multiple items variable items for
multiple users
so for that reason we have taken a
truncated length of 50 for a particular
user we are taking a list of 50 if it is
more than that we are just truncating if
it is less than that we are just padding
by some zeros ratings by some zero
values so this function essentially
correct function essentially take a
batch of the input batch and
ah based on the user item length it turn
kits or padded
okay the sale is almost covered 86
percent
just wait a minute
it will be done by after 25 seconds
okay now it is done so as I said you can
check any of the index you can check
that so this item user list is given for
a particular item who is the user and
what rating he has given for the
particular item
and let me process the trust information
quickly
that I have already told what is the
trash data frame is
for each of the user you have the
corresponding friend ID
and likewise here we are preparing the
user users item list and the user user
item list which would be required for
this architecture
then finally we are shaping the trust
information in a pickle file
so this is the basic graph data set
class the next I have already talked
about the collet function which is used
for padding and the trunk truncation
let me run these two cell
next I am coming to the components of
the architecture in order to implement
so we have already three component one
is the user model and is the item model
and is the rating prediction model and
there are some two common module which
is required for this architecture one is
the multi layer perception another is
the aggregator function
so if you see this architecture
let me show you here
yeah so the left one is the user
modeling item Metro rating prediction so
for these three architecture some common
thing is the aggregation and the multi
layer perception so this is to separate
so this multi layer perception is a
basically a sequential Network which is
you we have two linear ah layer and in
between two linear layer we have one
relu function for that and for the
aggregator function we have simple one
linear layer followed by one relu
function
next you know to implement the user
model component we have two kind of
thing one is the item aggregation and as
the social aggregation right so from the
item aggregation we will get one latent
vector and from the social aggregation
we will get one social Latin Factor n ah
lastly we'll concatenate these two to
get user latent factor so in the code
user you can see so we are taking user
embedding item embedding and data
embedding these three embedding has been
initialized by some random number
initially that I will show you quickly
then if you can see in the forward
function we are getting two kind of
value one is the h i i another is the h
i s so h i s is kind of social








In [ ]:
from IPython.display import Image
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
import torch
import torch.nn as nn
import random
import pickle
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [ ]:
rating_info = loadmat('/content/rating.mat')['rating']
trust_info = loadmat('/content/trustnetwork.mat')['trustnetwork']

In [ ]:
rating_info

array([[    1,     1,     3,     2],
       [    1,     2,     2,     2],
       [    1,     3,     3,     2],
       ...,
       [22166, 43538,     5,     5],
       [22166, 38711,     3,     4],
       [22166, 41790,     5,     3]], dtype=int32)

In [ ]:
trust_info

array([[15373,  9831],
       [ 4247,  9831],
       [ 4644,  9831],
       ...,
       [13181, 15645],
       [  897,  8000],
       [ 8000,   897]], dtype=uint16)

In [ ]:
rating_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0

In [ ]:
for x in rating_info:
    uid = x[0]
    iid = x[1]
    label = x[2]
    user_count = max(user_count, uid)
    item_count = max(item_count, iid)
    rate_count = max(rate_count, label)
    rating_list.append([uid, iid, label])

In [ ]:
pos_list = []
for x in rating_list:
	pos_list.append((x[0], x[1], x[2]))

In [ ]:
user_count, item_count, rate_count

(22166, 296277, 27)

In [ ]:
pos_list[0]

(1, 1, 3)

Prepare train, valid, test split of data

In [ ]:
pos_list = list(set(pos_list))
random.shuffle(pos_list)
num_test = int(len(pos_list) * 0.1)
test_set = pos_list[:num_test]
valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[2 * num_test:]
print('Train samples: {}, Valid samples: {}, Test samples: {}'.format(len(train_set), len(valid_set), len(test_set)))

Train samples: 730054, Valid samples: 91256, Test samples: 91256


Saving the data in a pickle file

In [ ]:
with open('./dataset_epinions.pkl', 'wb') as f:
	pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(valid_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
valid_df = pd.DataFrame(valid_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(test_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(rating_list, columns = ['uid', 'iid', 'label'])
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')

In [ ]:

train_df

,uid,iid,label
571500,1,6,11
306660,1,4,3
428699,1,9,3
497761,1,10,3
211124,1,1,3
...,...,...,...
586862,22166,21751,7
382484,22166,23442,5
492354,22166,41771,5
453739,22166,53125,5


In [ ]:

max(train_df['label'].values.tolist())

27

In [ ]:
min(train_df['label'].values.tolist())

1

Prepare user-item list

In [ ]:

for user in tqdm(range(user_count + 1)):
    user_df = train_df[train_df['uid'] == user]
    user_items = user_df['iid'].tolist()
    user_ratings = user_df['label'].tolist()
    if len(user_items) == 0:
        u_items_list.append([(0, 0)])
    else:
        u_items_list.append([(iid, rating) for iid, rating in zip(user_items, user_ratings)])

100%|██████████| 22167/22167 [00:18<00:00, 1180.17it/s]


In [ ]:
u_items_list[3]

[(57, 15),
 (54, 11),
 (48, 2),
 (52, 11),
 (44, 9),
 (51, 1),
 (42, 9),
 (55, 11),
 (46, 15),
 (50, 10),
 (49, 11),
 (53, 10),
 (45, 15),
 (43, 13)]

Prepare Item User List

In [ ]:
for item in tqdm(range(item_count + 1)):
    item_df = train_df[train_df['iid'] == item]
    item_users = item_df['uid'].tolist()
    item_ratings = item_df['label'].tolist()
    if len(item_users) == 0:
        i_users_list.append([(0, 0)])
    else:
        i_users_list.append([(uid, rating) for uid, rating in zip(item_users, item_ratings)])

100%|██████████| 296278/296278 [04:13<00:00, 1169.23it/s]


In [ ]:
i_users_list[2]

[(1, 2),
 (36, 2),
 (174, 2),
 (562, 2),
 (840, 2),
 (1196, 2),
 (1266, 2),
 (1351, 2),
 (1403, 2),
 (1729, 2),
 (2201, 2),
 (2564, 2),
 (2652, 2),
 (2779, 2),
 (3326, 2),
 (3334, 2),
 (3446, 2),
 (3615, 2),
 (3626, 2),
 (3939, 2),
 (4031, 2),
 (4138, 2),
 (4292, 2),
 (4330, 2),
 (4938, 2),
 (4961, 2),
 (4977, 2),
 (5113, 2),
 (5343, 2),
 (5646, 2),
 (5770, 2),
 (5843, 2),
 (5912, 2),
 (5939, 2),
 (6185, 2),
 (6245, 2),
 (6294, 2),
 (6326, 2),
 (6664, 2),
 (6752, 2),
 (6764, 2),
 (7030, 2),
 (7097, 2),
 (7145, 2),
 (7148, 2),
 (7216, 2),
 (7468, 2),
 (7525, 2),
 (7618, 2),
 (8419, 2),
 (8803, 2),
 (8810, 2),
 (8925, 2),
 (9054, 2),
 (9113, 2),
 (9432, 2),
 (9729, 2),
 (9872, 2),
 (10008, 2),
 (10138, 2),
 (10233, 2),
 (10329, 2),
 (11207, 2),
 (11505, 2),
 (11600, 2),
 (11801, 2),
 (12085, 2),
 (12107, 2),
 (12146, 2),
 (12181, 2),
 (12322, 2),
 (12647, 2),
 (12653, 2),
 (12807, 2),
 (14049, 2),
 (14401, 2),
 (14491, 2),
 (14580, 2),
 (14854, 2),
 (14937, 2),
 (15174, 2),
 (15307, 2),


Process trust info

In [ ]:
for x in trust_info:
    uid = x[0]
    fid = x[1]
    if uid > user_count or fid > user_count:
        continue
    trust_list.append([uid, fid])

In [ ]:
trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')


In [ ]:
trust_df

,uid,fid
283312,2,7659
84889,2,829
221672,2,21911
247685,2,7336
306637,2,6339
...,...,...
285484,22164,5864
283826,22164,11427
285362,22164,10897
135466,22165,2198


In [ ]:
for user in tqdm(range(user_count + 1)):
    user_df = trust_df[trust_df['uid'] == user]
    u_users = user_df['fid'].unique().tolist()
    if len(u_users) == 0:
        u_users_list.append([0])
        u_users_items_list.append([[(0, 0)]])
    else:
        u_users_list.append(u_users)
        uu_items = []
        for uid in u_users:
            uu_items.append(u_items_list[uid])
        u_users_items_list.append(uu_items)

100%|██████████| 22167/22167 [00:15<00:00, 1477.24it/s]


In [ ]:
with open('./list_epinions.pkl', 'wb') as f:
	pickle.dump(u_items_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(u_users_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(u_users_items_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(i_users_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump((user_count, item_count, rate_count), f, pickle.HIGHEST_PROTOCOL)

Prepare GraphDataset Class

In [ ]:

class GraphDataset(Dataset):
    def __init__(self, data, u_items_list, u_user_list, u_users_items_list, i_users_list):
        self.data = data
        self.u_items_list = u_items_list
        self.u_users_list = u_user_list
        self.u_users_items_list = u_users_items_list
        self.i_users_list = i_users_list

    def __getitem__(self, index):
        uid = self.data[index][0]
        iid = self.data[index][1]
        label = self.data[index][2]
        u_items = self.u_items_list[uid]
        u_users = self.u_users_list[uid]
        u_users_items = self.u_users_items_list[uid]
        i_users = self.i_users_list[iid]

        return (uid, iid, label), u_items, u_users, u_users_items, i_users

    def __len__(self):
        return len(self.data)

Collate function for padding and truncation

In [ ]:
truncate_len = 50

def collate_fn(batch_data):

    uids, iids, labels = [], [], []
    u_items, u_users, u_users_items, i_users = [], [], [], []
    u_items_len, u_users_len, i_users_len = [], [], []

    for data, u_items_u, u_users_u, u_users_items_u, i_users_i in batch_data:

        (uid, iid, label) = data
        uids.append(uid)
        iids.append(iid)
        labels.append(label)

        # user-items
        if len(u_items_u) <= truncate_len:
            u_items.append(u_items_u)
        else:
            u_items.append(random.sample(u_items_u, truncate_len))
        u_items_len.append(min(len(u_items_u), truncate_len))

        # user-users and user-users-items
        if len(u_users_u) <= truncate_len:
            u_users.append(u_users_u)
            u_u_items = []
            for uui in u_users_items_u:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            u_users_items.append(u_u_items)
        else:
            sample_index = random.sample(list(range(len(u_users_u))), truncate_len)
            u_users.append([u_users_u[si] for si in sample_index])

            u_users_items_u_tr = [u_users_items_u[si] for si in sample_index]
            u_u_items = []
            for uui in u_users_items_u_tr:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            u_users_items.append(u_u_items)

        u_users_len.append(min(len(u_users_u), truncate_len))

        # item-users
        if len(i_users_i) <= truncate_len:
            i_users.append(i_users_i)
        else:
            i_users.append(random.sample(i_users_i, truncate_len))
        i_users_len.append(min(len(i_users_i), truncate_len))

    batch_size = len(batch_data)

    # padding
    u_items_maxlen = max(u_items_len)
    u_users_maxlen = max(u_users_len)
    i_users_maxlen = max(i_users_len)

    u_item_pad = torch.zeros([batch_size, u_items_maxlen, 2], dtype=torch.long)
    for i, ui in enumerate(u_items):
        u_item_pad[i, :len(ui), :] = torch.LongTensor(ui)

    u_user_pad = torch.zeros([batch_size, u_users_maxlen], dtype=torch.long)
    for i, uu in enumerate(u_users):
        u_user_pad[i, :len(uu)] = torch.LongTensor(uu)

    u_user_item_pad = torch.zeros([batch_size, u_users_maxlen, u_items_maxlen, 2], dtype=torch.long)
    for i, uu_items in enumerate(u_users_items):
        for j, ui in enumerate(uu_items):
            u_user_item_pad[i, j, :len(ui), :] = torch.LongTensor(ui)

    i_user_pad = torch.zeros([batch_size, i_users_maxlen, 2], dtype=torch.long)
    for i, iu in enumerate(i_users):
        i_user_pad[i, :len(iu), :] = torch.LongTensor(iu)

    uids = torch.LongTensor(uids)
    iids = torch.LongTensor(iids)
    labels = torch.FloatTensor(labels)

    return uids, iids, labels, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad

Architecture components implemetaion

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, input_dim//2, bias=True),
            nn.ReLU(),
            nn.Linear(input_dim//2, output_dim, bias=True)
        )

    def forward(self, x):
        return self.mlp(x)

class Aggregator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Aggregator, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias=True),
            nn.ReLU()
        )

    def forward(self, x):
        return self.mlp(x)


UserModel Component

In [ ]:
class UserModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(UserModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb

        self.g_v = MLP(2*self.emb_dim, self.emb_dim)

        self.user_item_attn = MLP(2*self.emb_dim, 1)
        self.aggr_items = Aggregator(self.emb_dim, self.emb_dim)

        self.user_user_attn = MLP(2*self.emb_dim, 1)
        self.aggr_neighbors = Aggregator(self.emb_dim, self.emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10

    def forward(self, uids, u_item_pad, u_user_pad, u_user_item_pad):

        q_a = self.item_emb(u_item_pad[:,:,0])
        u_item_er = self.rating_emb(u_item_pad[:,:,1])
        x_ia = self.g_v(torch.cat([q_a, u_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a.size())
        mask_u = torch.where(u_item_pad[:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i = mask_u.unsqueeze(2).expand_as(x_ia) * self.user_emb(uids).unsqueeze(1).expand_as(x_ia)
        alpha = self.user_item_attn(torch.cat([x_ia, p_i], dim=2).view(-1, 2*self.emb_dim)).view(mask_u.size())
        alpha = torch.exp(alpha)*mask_u
        alpha = alpha / (torch.sum(alpha, 1).unsqueeze(1).expand_as(alpha) + self.eps)
        h_iI = self.aggr_items(torch.sum(alpha.unsqueeze(2).expand_as(x_ia) * x_ia, 1))


        q_a_s = self.item_emb(u_user_item_pad[:,:,:,0])
        u_user_item_er = self.rating_emb(u_user_item_pad[:,:,:,1])
        x_ia_s = self.g_v(torch.cat([q_a_s, u_user_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a_s.size())
        mask_s = torch.where(u_user_item_pad[:,:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i_s = mask_s.unsqueeze(3).expand_as(x_ia_s) * self.user_emb(u_user_pad).unsqueeze(2).expand_as(x_ia_s)
        alpha_s = self.user_item_attn(torch.cat([x_ia_s, p_i_s], dim=3).view(-1, 2*self.emb_dim)).view(mask_s.size())
        alpha_s = torch.exp(alpha_s)*mask_s
        alpha_s = alpha_s / (torch.sum(alpha_s, 2).unsqueeze(2).expand_as(alpha_s) + self.eps)
        h_oI_temp = torch.sum(alpha_s.unsqueeze(3).expand_as(x_ia_s) * x_ia_s, 2)
        h_oI = self.aggr_items(h_oI_temp.view(-1, self.emb_dim)).view(h_oI_temp.size())

        beta = self.user_user_attn(torch.cat([h_oI, self.user_emb(u_user_pad)], dim = 2).view(-1, 2 * self.emb_dim)).view(u_user_pad.size())
        mask_su = torch.where(u_user_pad > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        beta = torch.exp(beta) * mask_su
        beta = beta / (torch.sum(beta, 1).unsqueeze(1).expand_as(beta) + self.eps)
        h_iS = self.aggr_neighbors(torch.sum(beta.unsqueeze(2).expand_as(h_oI) * h_oI, 1))

        h_i = self.mlp(torch.cat([h_iI, h_iS], dim = 1))

        return h_i


ItemModel Component

In [ ]:
class ItemModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(ItemModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb

        self.g_u = MLP(2*self.emb_dim, self.emb_dim)

        self.item_users_attn = MLP(2*self.emb_dim, 1)
        self.aggr_users = Aggregator(self.emb_dim, self.emb_dim)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10

    def forward(self, iids, i_user_pad):

        p_t = self.user_emb(i_user_pad[:,:,0])
        i_user_er = self.rating_emb(i_user_pad[:,:,1])
        mask_i = torch.where(i_user_pad[:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        f_jt = self.g_u(torch.cat([p_t, i_user_er], dim = 2).view(-1, 2 * self.emb_dim)).view(p_t.size())
        q_j = mask_i.unsqueeze(2).expand_as(f_jt) * self.item_emb(iids).unsqueeze(1).expand_as(f_jt)
        mu_jt = self.item_users_attn(torch.cat([f_jt, q_j], dim = 2).view(-1, 2 * self.emb_dim)).view(mask_i.size())
        mu_jt = torch.exp(mu_jt) * mask_i
        mu_jt = mu_jt / (torch.sum(mu_jt, 1).unsqueeze(1).expand_as(mu_jt) + self.eps)

        z_j = self.aggr_users(torch.sum(mu_jt.unsqueeze(2).expand_as(f_jt) * f_jt, 1))

        return z_j

Reccomendation Module

In [ ]:
class RecModule(nn.Module):
    def __init__(self, n_users, n_items, n_ratings, emb_dim = 64):
        super(RecModule, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.n_ratings = n_ratings
        self.emb_dim = emb_dim

        self.user_emb = nn.Embedding(self.n_users, self.emb_dim, padding_idx=0)
        self.item_emb = nn.Embedding(self.n_items, self.emb_dim, padding_idx=0)
        self.rating_emb = nn.Embedding(self.n_ratings, self.emb_dim, padding_idx=0)

        self.user_model = UserModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)
        self.item_model = ItemModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias=True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias=True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, 1)
        )

    def forward(self, uids, iids, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad):

        h_i = self.user_model(uids, u_item_pad, u_user_pad, u_user_item_pad)
        z_j = self.item_model(iids, i_user_pad)

        r_ij = self.mlp(torch.cat([h_i, z_j], dim=1))

        return r_ij

Set Hyperparameters

In [ ]:



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device - ' + str(device))
batch_size = 128
embed_dim = 64
learning_rate = 0.001
n_epochs = 5

device - cpu


In [ ]:
with open('./dataset_epinions.pkl', 'rb') as f:
    train_set = pickle.load(f)
    valid_set = pickle.load(f)
    test_set = pickle.load(f)

with open('./list_epinions.pkl', 'rb') as f:
    u_items_list = pickle.load(f)
    u_users_list = pickle.load(f)
    u_users_items_list = pickle.load(f)
    i_users_list = pickle.load(f)
    (user_count, item_count, rate_count) = pickle.load(f)

In [ ]:
train_data = GraphDataset(train_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
valid_data = GraphDataset(valid_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
test_data = GraphDataset(test_set, u_items_list, u_users_list, u_users_items_list, i_users_list)

Prepare DataLoader

In [ ]:
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)

In [ ]:

len(train_loader)

5704

In [ ]:



for i in train_loader:
    for j in i:
        print(j)
    break

tensor([ 6877, 14677, 18530,    18, 14523, 11069, 15497, 20416, 11267, 19017,
        10877,  7961, 14287, 20331, 19041,  2444,  5219,  9414,  4236,   214,
        21170,  7534, 15534, 21092,  7862,  3203,  2524, 13728, 13313,  3558,
         2269,  6019, 15893,  2712, 11815, 13262, 21699, 15355, 11880, 13533,
        10022,  2760, 15965, 10948, 13151, 12856,  3374, 10709,  5131,  8315,
        13533, 19697, 16017, 14084,  3168, 17775, 16750,  9174, 17131,  6580,
         7942, 20891, 20293, 10123,  1914, 16836,  8416,  7515, 18766,  5278,
        18204,  6877,  8282, 16091, 13367, 16204, 16059,  4378, 13451,  2461,
        19447,  9222,  5131,  1105,  5270,  5289,  9739,  8884,  2344, 16545,
        14187,  7941,  2668, 22156, 11790,  3240, 14333, 20026, 15973,  5287,
         3783, 22152,  7246,  4319, 15965,  5267, 19766,  6877,  5657,  4691,
        13510,   796,  3018,   112, 17209,  1943, 14596,  4504, 15534,  2318,
        15162,  8533,  8890,  9072, 11985, 12022, 12993, 21134])

In [ ]:
model = RecModule(user_count+1, item_count+1, rate_count+1, embed_dim).to(device)

In [ ]:

optimizer = torch.optim.RMSprop(model.parameters(), learning_rate)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 4, gamma = 0.1)


Training

In [ ]:
for epoch in range(n_epochs):

    # Training step
    model.train()
    s_loss = 0
    for i, (uids, iids, labels, u_items, u_users, u_users_items, i_users) in tqdm(enumerate(train_loader), total=len(train_loader)):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)

        optimizer.zero_grad()
        outputs = model(uids, iids, u_items, u_users, u_users_items, i_users)
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        s_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

    # Validate step
    model.eval()
    errors = []
    with torch.no_grad():
        for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(valid_loader):
            uids = uids.to(device)
            iids = iids.to(device)
            labels = labels.to(device)
            u_items = u_items.to(device)
            u_users = u_users.to(device)
            u_users_items = u_users_items.to(device)
            i_users = i_users.to(device)
            preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
            error = torch.abs(preds.squeeze(1) - labels)
            errors.extend(error.data.cpu().numpy().tolist())

    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(np.power(errors, 2)))

    scheduler.step()

    ckpt_dict = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    torch.save(ckpt_dict, './latest_checkpoint.pth')

    if epoch == 0:
        best_mae = mae
    elif mae < best_mae:
        best_mae = mae
        torch.save(ckpt_dict, './best_checkpoint_{}.pth'.format(embed_dim))

    print('Epoch {} validation: MAE: {:.4f}, RMSE: {:.4f}, Best MAE: {:.4f}'.format(epoch+1, mae, rmse, best_mae))

100%|██████████| 713/713 [16:30<00:00,  1.39s/it]


Epoch 1 validation: MAE: 1.9217, RMSE: 4.3973, Best MAE: 1.9217


 57%|█████▋    | 3272/5704 [2:58:43<2:14:47,  3.33s/it]

Test The Model

In [ ]:



embed_dim = 64
checkpoint = torch.load('./latest_checkpoint.pth'.format(embed_dim))
model = RecModule(user_count+1, item_count+1, rate_count+1, embed_dim).to(device)
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
model.eval()
test_errors = []
with torch.no_grad():
    for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(test_loader):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)
        preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
        error = torch.abs(preds.squeeze(1) - labels)
        test_errors.extend(error.data.cpu().numpy().tolist())

test_mae = np.mean(test_errors)
test_rmse = np.sqrt(np.mean(np.power(test_errors, 2)))
print('Test: MAE: {:.4f}, RMSE: {:.4f}'.format(test_mae, test_rmse))